In [1]:
from utils import RL_with_constraint, RL_without_constraint, RLWKC_with_constraint, RLWKC_without_constraint, HTP, sparsePermutation, addNoise, meanNormalizedError, sgn, one_norm, only_priors
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from IPython.display import clear_output
import random  
import scipy
np.random.seed(90)
random.seed(90)

In [2]:
n_Permuted = 16                                           # number of permutation
cv_measurements = 5
m = 32                                                    # number of known correspondences
noise_fraction = 0.02
permutation_runs, noise_runs = 2, 2
p = 240                                                   # dimension of x
k = 14                                                    # sparsity of x
N_Metrics = np.zeros(shape = (permutation_runs, noise_runs, 9))
x = np.load("x_sparsity_" + str(k) + ".npy")
A_cv =  np.load("A_cv_fixed.npy")  
y_cv = addNoise(A_cv @ x, noise_fraction)     


for N in [100]:
    A = np.load("A_fixed.npy")[:N, :]           # extract N rows from the generated matrix

    for per_run in range(permutation_runs):
        print("N: {}, Permutation run: {}".format(N, per_run))
        P = sparsePermutation(N, m, n_Permuted)
        noiseLess_y = P @ A @ x
        z_true = noiseLess_y - A @ x
        max_abs_z = np.max(np.abs(z_true))
        sum_abs_z = np.sum(np.abs(z_true))

        l1_RLWKC_with_constraint, l2_RLWKC_with_constraint = None, None
        l1_RLWKC_without_constraint, l2_RLWKC_without_constraint = None, None
        l1_RL_with_constraint, l2_RL_with_constraint = None, None
        l1_RL_without_constraint, l2_RL_without_constraint = None, None
        l1_one_norm = None
        l1_only_priors = None

        for noise_run in range(noise_runs):
            print("N: {}, Noisy run: {}".format(k, noise_run))
            noisy_y = addNoise(noiseLess_y, noise_fraction)
            xhat_RLWKC_with_constraint, l1_RLWKC_with_constraint, l2_RLWKC_with_constraint = RLWKC_with_constraint(A, noisy_y, A_cv, y_cv, m, l1_RLWKC_with_constraint, l2_RLWKC_with_constraint)
            xhat_RLWKC_without_constraint, l1_RLWKC_without_constraint, l2_RLWKC_without_constraint = RLWKC_without_constraint(A, noisy_y, A_cv, y_cv, m, l1_RLWKC_without_constraint, l2_RLWKC_without_constraint)
            xhat_RL_with_constraint, l1_RL_with_constraint, l2_RL_with_constraint = RL_with_constraint(A, noisy_y, A_cv, y_cv, l1_RL_with_constraint, l2_RL_with_constraint)
            xhat_RL_without_constraint, l1_RL_without_constraint, l2_RL_without_constraint = RL_without_constraint(A, noisy_y, A_cv, y_cv, l1_RL_without_constraint, l2_RL_without_constraint)
            xhat_HTP = HTP(np.concatenate((A, A_cv), axis = 0), np.concatenate((noisy_y, y_cv), axis = 0), k)
            xhat_one_norm, l1_one_norm = one_norm(A, noisy_y, A_cv, y_cv, l1_one_norm)
            xhat_only_priors, l1_only_priors = only_priors(A, noisy_y, A_cv, y_cv, m, l1_only_priors)
  
            N_Metrics[per_run][noise_run][0] = meanNormalizedError(x, xhat_RLWKC_with_constraint)
            N_Metrics[per_run][noise_run][1] = meanNormalizedError(x, xhat_RLWKC_without_constraint)
            N_Metrics[per_run][noise_run][2] = meanNormalizedError(x, xhat_RL_with_constraint)
            N_Metrics[per_run][noise_run][3] = meanNormalizedError(x, xhat_RL_without_constraint)
            N_Metrics[per_run][noise_run][4] = meanNormalizedError(x, xhat_HTP)
            N_Metrics[per_run][noise_run][5] = meanNormalizedError(x, xhat_one_norm)
            N_Metrics[per_run][noise_run][6] = meanNormalizedError(x, xhat_only_priors)
            N_Metrics[per_run][noise_run][7] = max_abs_z
            N_Metrics[per_run][noise_run][8] = sum_abs_z
           

            print(N_Metrics[per_run][noise_run][0], N_Metrics[per_run][noise_run][1], N_Metrics[per_run][noise_run][2], N_Metrics[per_run][noise_run][3], N_Metrics[per_run][noise_run][4], N_Metrics[per_run][noise_run][5], N_Metrics[per_run][noise_run][6], N_Metrics[per_run][noise_run][7], N_Metrics[per_run][noise_run][8])

    np.save('N_Metrics' + str(N) + '.npy', N_Metrics)

N: 100, Permutation run: 0
N: 14, Noisy run: 0
Choose lambda1 properly
0.02708081933380165 0.033994443682122774 0.14946892653293123 0.1699789703471624 0.6349609652983684 0.9951813712115958 0.7241718539354522 14.258056767165815 104.07095398139649
N: 14, Noisy run: 1
Choose lambda1 properly
0.03821245763695885 0.0389042003842905 0.16700812890101632 0.16264045893221288 0.7316049237206522 0.9889276130571298 0.7754362206285683 14.258056767165815 104.07095398139649
N: 14, Noisy run: 2
Choose lambda1 properly
0.02163987029094612 0.027942868161440627 0.190859287245002 0.18666320295708919 0.7080450249083156 0.9973058412841701 0.7795946316123465 14.258056767165815 104.07095398139649
N: 14, Noisy run: 3
Choose lambda1 properly
0.029065609634100466 0.03685327940016352 0.16387053715824448 0.1819129531611079 0.6381198952541288 0.9902359213674053 0.6197370325030519 14.258056767165815 104.07095398139649
N: 14, Noisy run: 4
Choose lambda1 properly
0.043014997361904526 0.049835241325025 0.16256506874216